In [7]:
import os
import re
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
from google.colab import files

[PD] OK=10, SKIP=0
[ET] OK=8, SKIP=0
Готово: /content/PD_ET_csv.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
COLUMN_NAMES= ['EOG_left_h', 'EOG_right_h', 'EOG_left_v', 'EOG_right_v',
    'head_horizontal', 'head_frontal', 'head_sagittal', 'Сигнал_8',
    'Сигнал_9', 'Сигнал_10', 'Сигнал_11']

COLUMN_NAMES= ['EOG_left_h', 'EOG_right_h', 'EOG_left_v', 'EOG_right_v',
    'head_horizontal', 'head_frontal', 'head_sagittal', 'Сигнал_8', 'Сигнал_9']

In [ ]:
def _nat_key(p: Path):
    s = p.stem
    try:
        return (0, int(s))
    except ValueError:
        return (1, s.lower())

In [ ]:
def parse_data_to_columns(file_path: Path) -> pd.DataFrame:
    pattern= re.compile(r'data\s*=\s*\[(.*?)\]\s*;?', flags=re.DOTALL)
    try:
        text= file_path.read_text(encoding='cp1251', errors='strict')
    except Exception:
        text= file_path.read_text(encoding='utf-8', errors='ignore')

    data_lists= []
    for m in pattern.finditer(text):
        data_str= m.group(1).replace("\n", " ").replace("\r", " ")
        parts= [x.strip() for x in data_str.split(",") if x.strip()]
        try:
            nums= [float(x) for x in parts]
        except ValueError:
            continue
        data_lists.append(nums)

    if not data_lists:
        return pd.DataFrame()

    # skip first two duee to the
    if len(data_lists)>= 2:
        data_lists= data_lists[2:]
    else:
        return pd.DataFrame()

    first_len= len(data_lists[0])
    cols= {}
    for idx, data in enumerate(data_lists, start=1):
        col_name= f"column {idx}"
        if len(data)< first_len:
            padded= data + [float("nan")]*(first_len-len(data))
        else:
            padded= data[:first_len]
        cols[col_name]= padded

    return pd.DataFrame(cols)

In [ ]:
def process_group(group_name: str, in_root: Path, out_root: Path):
    in_dir = in_root / group_name
    out_dir = out_root / group_name
    out_dir.mkdir(parents=True, exist_ok=True)

    txt_files = sorted(in_dir.glob("*.txt"), key=_nat_key)
    if not txt_files:
        print(f"no ,txt in {in_dir}")
        return

    ok, skip = 0, 0
    for fp in txt_files:
        df = parse_data_to_columns(fp)
        if df.empty:
            print(f"[SKIP] {group_name}/{fp.name} couldnt")
            skip += 1
            continue

        if df.shape[1] != len(COLUMN_NAMES):
            print(f"[SKIP] {group_name}/{fp.name}: found column {df.shape[1]}, eexpected {len(COLUMN_NAMES)}")
            skip += 1
            continue

        df.columns = COLUMN_NAMES
        df.insert(0, "time", range(len(df)))

        miss = [c for c in EXPORT_COLS if c not in df.columns]
        if miss:
            raise ValueError(f"{group_name}/{fp.name}: lacks columns {miss}")

        out_csv = out_dir / f"{fp.stem}.csv"
        df.loc[:, EXPORT_COLS].to_csv(out_csv, index=False)
        ok += 1

    print(f"[{group_name}] OK={ok}, SKIP={skip}")

In [ ]:
IN_ROOT = Path("/content")
OUT_ROOT = Path("/content/PD_ET_csv")

if OUT_ROOT.exists():
    shutil.rmtree(OUT_ROOT)
OUT_ROOT.mkdir(parents=True, exist_ok=True)

process_group("PD", IN_ROOT, OUT_ROOT)
process_group("ET", IN_ROOT, OUT_ROOT)

In [ ]:
zip_base = str(OUT_ROOT)
zip_path = shutil.make_archive(zip_base, "zip", str(OUT_ROOT))

print(zip_path)
files.download(zip_path)